In [ ]:
#@title 🎧 Download Narration Audio & Play Introduction
import os as _os
if not _os.path.exists("/content/narration"):
    !pip install -q gdown
    import gdown
    gdown.download(id="1_yOuaRupWcvvBB5tNnjVrtDllXqg6x4Q", output="/content/narration.zip", quiet=False)
    !unzip -q /content/narration.zip -d /content/narration
    !rm /content/narration.zip
    print(f"Loaded {len(_os.listdir('/content/narration'))} narration segments")
else:
    print("Narration audio already loaded.")

from IPython.display import Audio, display
display(Audio("/content/narration/03_00_intro.mp3"))


In [ ]:
#@title 🎧 Code Walkthrough: Setup
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_01_setup.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 What to Look For: Visualize Landscape
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_04_visualize_landscape.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

In [ ]:
#@title 🎧 Listen: Why It Matters
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_02_why_it_matters.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


# Optimization: SGD, Momentum, and Adam -- Vizuara

> **What you will build:** Three optimizers from scratch -- SGD, SGD with Momentum, and Adam -- and watch them compete on a real loss landscape.

## 1. Why Does This Matter?

The optimizer is the engine of learning. It takes the gradients computed during backpropagation and uses them to update the model's weights. A bad optimizer means the model learns slowly, gets stuck in poor solutions, or diverges entirely.

The history of deep learning optimizers tells a clear story: **SGD** is simple but slow. **Momentum** adds acceleration. **Adam** adapts the learning rate per-weight. Understanding this progression is essential because Adam (and its variant AdamW) is used in virtually every modern LLM.

In this notebook, we will implement all three from scratch, trace through them numerically, and watch them compete.

In [ ]:
#@title 🎧 Listen: Building Intuition
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_03_building_intuition.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 2. Building Intuition

Imagine you are blindfolded in a hilly landscape, trying to find the lowest valley. You can feel the slope beneath your feet (the gradient), but you cannot see where you are going.

**SGD:** You take a fixed-size step downhill at every moment. Simple, but you zigzag wildly in narrow valleys.

**Momentum:** You roll a ball downhill. It builds up speed in consistent directions and smooths out oscillations.

**Adam:** You have a smart GPS that tracks both the average slope and how much it has been changing. It tells you exactly how big a step to take for each direction independently.

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

# Create a loss landscape: a narrow valley (Rosenbrock-like)
def loss_fn(x, y):
    """A narrow valley loss landscape -- hard for SGD, easier for Adam."""
    return (1 - x)**2 + 100 * (y - x**2)**2

# Visualize the landscape
x = np.linspace(-2, 2, 200)
y = np.linspace(-1, 3, 200)
X, Y = np.meshgrid(x, y)
Z = loss_fn(X, Y)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# 3D view
ax1 = fig.add_subplot(121, projection='3d')
ax1.plot_surface(X, Y, np.log1p(Z), cmap='viridis', alpha=0.8, linewidth=0)
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_zlabel('log(1 + Loss)')
ax1.set_title('Loss Landscape (3D)', fontweight='bold')

# Contour view
ax2 = fig.add_subplot(122)
contour = ax2.contour(X, Y, np.log1p(Z), levels=30, cmap='viridis')
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_title('Loss Landscape (Contours)', fontweight='bold')
ax2.plot(1, 1, 'r*', markersize=15, label='Minimum (1,1)')
ax2.legend()

plt.tight_layout()
plt.savefig('loss_landscape.png', dpi=150, bbox_inches='tight')
plt.show()
print("Checkpoint: The minimum is at (1, 1). The narrow valley makes optimization hard.")

In [ ]:
#@title 🎧 Listen: The Mathematics
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_05_the_mathematics.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 3. The Mathematics

### Stochastic Gradient Descent (SGD)

The simplest update rule:

$$\theta_{t+1} = \theta_t - \eta \nabla L(\theta_t)$$

where $\eta$ is the learning rate and $\nabla L$ is the gradient.

### SGD with Momentum

Add a velocity term that accumulates past gradients:

$$v_{t+1} = \beta v_t + \nabla L(\theta_t)$$
$$\theta_{t+1} = \theta_t - \eta \, v_{t+1}$$

where $\beta$ is the momentum coefficient (typically 0.9).

### Adam (Adaptive Moment Estimation)

Maintain two running statistics per weight:

**First moment** (mean of gradients):
$$m_t = \beta_1 m_{t-1} + (1 - \beta_1) g_t$$

**Second moment** (mean of squared gradients):
$$v_t = \beta_2 v_{t-1} + (1 - \beta_2) g_t^2$$

**Bias correction:**
$$\hat{m}_t = \frac{m_t}{1 - \beta_1^t}, \quad \hat{v}_t = \frac{v_t}{1 - \beta_2^t}$$

**Update:**
$$\theta_t = \theta_{t-1} - \eta \frac{\hat{m}_t}{\sqrt{\hat{v}_t} + \epsilon}$$

The key insight: dividing by $\sqrt{\hat{v}_t}$ makes the effective step size **smaller** for weights with large, variable gradients and **larger** for weights with small, consistent gradients.

In [ ]:
#@title 🎧 Code Walkthrough: Numerical Trace
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_06_numerical_trace.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Numerical trace: one step of each optimizer
theta = 0.5
gradient = 0.2
lr = 0.01

# SGD
theta_sgd = theta - lr * gradient
print(f"SGD:      theta = {theta} - {lr} * {gradient} = {theta_sgd}")

# Momentum (first step, v0 = 0)
beta = 0.9
v = 0.0
v_new = beta * v + gradient
theta_mom = theta - lr * v_new
print(f"Momentum: v = {beta}*{v} + {gradient} = {v_new}")
print(f"          theta = {theta} - {lr} * {v_new} = {theta_mom}")

# Adam (first step, m0 = 0, v0 = 0)
beta1, beta2, eps = 0.9, 0.999, 1e-8
m = (1 - beta1) * gradient  # 0.1 * 0.2 = 0.02
v_adam = (1 - beta2) * gradient**2  # 0.001 * 0.04 = 0.00004
m_hat = m / (1 - beta1**1)  # 0.02 / 0.1 = 0.2
v_hat = v_adam / (1 - beta2**1)  # 0.00004 / 0.001 = 0.04
theta_adam = theta - 0.001 * m_hat / (np.sqrt(v_hat) + eps)
print(f"Adam:     m_hat = {m_hat:.4f}, v_hat = {v_hat:.4f}")
print(f"          theta = {theta} - 0.001 * {m_hat:.4f} / ({np.sqrt(v_hat):.4f}) = {theta_adam:.6f}")

In [ ]:
#@title 🎧 Transition: Build Sgd Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_07_build_sgd_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 4. Let's Build It -- Component by Component

### 4.1 SGD from Scratch

In [ ]:
#@title 🎧 Code Walkthrough: Sgd From Scratch
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_08_sgd_from_scratch.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
class SGD:
    """Stochastic Gradient Descent optimizer."""

    def __init__(self, params, lr=0.01):
        self.params = list(params)
        self.lr = lr

    def zero_grad(self):
        for p in self.params:
            if p.grad is not None:
                p.grad.zero_()

    def step(self):
        for p in self.params:
            if p.grad is not None:
                p.data -= self.lr * p.grad.data

# Test: minimize f(x) = (x - 3)^2
x = torch.tensor([0.0], requires_grad=True)
optimizer = SGD([x], lr=0.1)

trajectory = [x.item()]
for step in range(20):
    loss = (x - 3)**2
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    trajectory.append(x.item())

print(f"SGD: x went from {trajectory[0]:.3f} to {trajectory[-1]:.3f} (target: 3.0)")
print(f"Trajectory: {[f'{t:.2f}' for t in trajectory[:6]]}...")

In [ ]:
#@title 🎧 Listen: Momentum Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_09_momentum_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.2 SGD with Momentum from Scratch

In [ ]:
#@title 🎧 Code Walkthrough: Sgdmomentum From Scratch
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_10_sgdmomentum_from_scratch.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
class SGDMomentum:
    """SGD with momentum optimizer."""

    def __init__(self, params, lr=0.01, momentum=0.9):
        self.params = list(params)
        self.lr = lr
        self.momentum = momentum
        # Initialize velocity for each parameter
        self.velocities = [torch.zeros_like(p.data) for p in self.params]

    def zero_grad(self):
        for p in self.params:
            if p.grad is not None:
                p.grad.zero_()

    def step(self):
        for i, p in enumerate(self.params):
            if p.grad is not None:
                # Update velocity
                self.velocities[i] = self.momentum * self.velocities[i] + p.grad.data
                # Update parameter
                p.data -= self.lr * self.velocities[i]

# Test: minimize f(x) = (x - 3)^2
x = torch.tensor([0.0], requires_grad=True)
optimizer = SGDMomentum([x], lr=0.1, momentum=0.9)

trajectory = [x.item()]
for step in range(20):
    loss = (x - 3)**2
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    trajectory.append(x.item())

print(f"Momentum: x went from {trajectory[0]:.3f} to {trajectory[-1]:.3f} (target: 3.0)")
print(f"Trajectory: {[f'{t:.2f}' for t in trajectory[:6]]}...")
print("Notice: Momentum overshoots then corrects -- it oscillates around the target!")

In [ ]:
#@title 🎧 Listen: Adam Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_11_adam_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.3 Adam from Scratch

In [ ]:
#@title 🎧 Code Walkthrough: Adam From Scratch
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_12_adam_from_scratch.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
class Adam:
    """Adam optimizer (Adaptive Moment Estimation)."""

    def __init__(self, params, lr=0.001, betas=(0.9, 0.999), eps=1e-8):
        self.params = list(params)
        self.lr = lr
        self.beta1, self.beta2 = betas
        self.eps = eps
        self.t = 0  # Step counter for bias correction

        # Initialize moments for each parameter
        self.m = [torch.zeros_like(p.data) for p in self.params]  # First moment
        self.v = [torch.zeros_like(p.data) for p in self.params]  # Second moment

    def zero_grad(self):
        for p in self.params:
            if p.grad is not None:
                p.grad.zero_()

    def step(self):
        self.t += 1
        for i, p in enumerate(self.params):
            if p.grad is not None:
                g = p.grad.data

                # Update first moment (mean of gradients)
                self.m[i] = self.beta1 * self.m[i] + (1 - self.beta1) * g

                # Update second moment (mean of squared gradients)
                self.v[i] = self.beta2 * self.v[i] + (1 - self.beta2) * g**2

                # Bias correction
                m_hat = self.m[i] / (1 - self.beta1**self.t)
                v_hat = self.v[i] / (1 - self.beta2**self.t)

                # Update parameters
                p.data -= self.lr * m_hat / (torch.sqrt(v_hat) + self.eps)

# Test: minimize f(x) = (x - 3)^2
x = torch.tensor([0.0], requires_grad=True)
optimizer = Adam([x], lr=0.1)

trajectory = [x.item()]
for step in range(50):
    loss = (x - 3)**2
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    trajectory.append(x.item())

print(f"Adam: x went from {trajectory[0]:.3f} to {trajectory[-1]:.3f} (target: 3.0)")
print(f"Trajectory: {[f'{t:.2f}' for t in trajectory[:8]]}...")

In [ ]:
#@title 🎧 Listen: 1d Comparison Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_13_1d_comparison_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### Visualization: Comparing Optimizer Trajectories on 1D

In [ ]:
#@title 🎧 What to Look For: Visualize 1d Comparison
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_14_visualize_1d_comparison.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))

# Track all three optimizers on f(x) = (x - 3)^2
trajectories = {}

for OptClass, name, lr, kwargs in [
    (SGD, 'SGD', 0.1, {}),
    (SGDMomentum, 'SGD + Momentum', 0.1, {'momentum': 0.9}),
    (Adam, 'Adam', 0.1, {}),
]:
    x = torch.tensor([0.0], requires_grad=True)
    opt = OptClass([x], lr=lr, **kwargs)
    traj = [x.item()]
    for step in range(30):
        loss = (x - 3)**2
        loss.backward()
        opt.step()
        opt.zero_grad()
        traj.append(x.item())
    trajectories[name] = traj

colors = {'SGD': '#e74c3c', 'SGD + Momentum': '#3498db', 'Adam': '#2ecc71'}
for name, traj in trajectories.items():
    ax.plot(traj, label=name, linewidth=2, color=colors[name], marker='o', markersize=3)

ax.axhline(y=3.0, color='black', linestyle='--', linewidth=1, alpha=0.5, label='Target (x=3)')
ax.set_xlabel('Step', fontsize=12)
ax.set_ylabel('x value', fontsize=12)
ax.set_title('Optimizer Comparison: Minimizing f(x) = (x - 3)^2', fontsize=13, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('optimizer_comparison_1d.png', dpi=150, bbox_inches='tight')
plt.show()
print("Checkpoint: SGD converges linearly, Momentum overshoots, Adam adapts.")

In [ ]:
#@title 🎧 Before You Start: Todo1 Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_15_todo1_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 5. Your Turn

### TODO 1: Implement AdamW (Adam with Decoupled Weight Decay)

Standard Adam applies weight decay through the gradient (L2 regularization). AdamW decouples weight decay from the gradient update, applying it directly to the weights.

In [ ]:
#@title 🎧 Before You Start: Todo1 Adamw
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_16_todo1_adamw.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
class AdamW:
    """Adam with decoupled weight decay (used in every modern LLM)."""

    def __init__(self, params, lr=0.001, betas=(0.9, 0.999), eps=1e-8, weight_decay=0.01):
        self.params = list(params)
        self.lr = lr
        self.beta1, self.beta2 = betas
        self.eps = eps
        self.weight_decay = weight_decay
        self.t = 0

        self.m = [torch.zeros_like(p.data) for p in self.params]
        self.v = [torch.zeros_like(p.data) for p in self.params]

    def zero_grad(self):
        for p in self.params:
            if p.grad is not None:
                p.grad.zero_()

    def step(self):
        self.t += 1
        for i, p in enumerate(self.params):
            if p.grad is not None:
                g = p.grad.data

                # TODO: Update first moment (m)
                # self.m[i] = ...  # YOUR CODE HERE

                # TODO: Update second moment (v)
                # self.v[i] = ...  # YOUR CODE HERE

                # TODO: Bias correction
                # m_hat = ...  # YOUR CODE HERE
                # v_hat = ...  # YOUR CODE HERE

                # TODO: Adam update (WITHOUT weight decay)
                # p.data -= self.lr * m_hat / (torch.sqrt(v_hat) + self.eps)

                # TODO: Apply DECOUPLED weight decay
                # p.data -= ...  # YOUR CODE HERE
                pass

    # SOLUTION (uncomment to verify):
    # def step(self):
    #     self.t += 1
    #     for i, p in enumerate(self.params):
    #         if p.grad is not None:
    #             g = p.grad.data
    #             self.m[i] = self.beta1 * self.m[i] + (1 - self.beta1) * g
    #             self.v[i] = self.beta2 * self.v[i] + (1 - self.beta2) * g**2
    #             m_hat = self.m[i] / (1 - self.beta1**self.t)
    #             v_hat = self.v[i] / (1 - self.beta2**self.t)
    #             p.data -= self.lr * m_hat / (torch.sqrt(v_hat) + self.eps)
    #             p.data -= self.lr * self.weight_decay * p.data

In [ ]:
#@title 🎧 Before You Start: Todo2 Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_17_todo2_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### TODO 2: Trace Through Adam Manually

Given: $\theta_0 = 1.0$, $m_0 = 0$, $v_0 = 0$, $\beta_1 = 0.9$, $\beta_2 = 0.999$, $\eta = 0.01$, $\epsilon = 10^{-8}$

Gradients over 3 steps: $g_1 = 0.5$, $g_2 = -0.3$, $g_3 = 0.4$

In [ ]:
#@title 🎧 Before You Start: Todo2 Manual Trace
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_18_todo2_manual_trace.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# TODO: Compute m, v, m_hat, v_hat, and theta for each step
# Fill in the values below

# Step 1: g = 0.5
# m1 = 0.9 * 0 + 0.1 * 0.5 = ???
# v1 = 0.999 * 0 + 0.001 * 0.5^2 = ???
# m_hat1 = m1 / (1 - 0.9^1) = ???
# v_hat1 = v1 / (1 - 0.999^1) = ???
# theta1 = 1.0 - 0.01 * m_hat1 / (sqrt(v_hat1) + 1e-8) = ???

# Step 2: g = -0.3
# m2 = ???
# v2 = ???
# ...

# Step 3: g = 0.4
# m3 = ???
# ...

print("Verify your manual calculation against the code:")
theta = 1.0
m, v = 0.0, 0.0
beta1, beta2, lr, eps = 0.9, 0.999, 0.01, 1e-8
grads = [0.5, -0.3, 0.4]

for t, g in enumerate(grads, 1):
    m = beta1 * m + (1 - beta1) * g
    v = beta2 * v + (1 - beta2) * g**2
    m_hat = m / (1 - beta1**t)
    v_hat = v / (1 - beta2**t)
    theta = theta - lr * m_hat / (np.sqrt(v_hat) + eps)
    print(f"Step {t}: g={g:+.1f}, m_hat={m_hat:.6f}, v_hat={v_hat:.6f}, theta={theta:.6f}")

In [ ]:
#@title 🎧 Transition: 2d Race Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_19_2d_race_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 6. Putting It All Together: 2D Optimizer Race

Let us race all three optimizers on a challenging 2D loss surface.

In [ ]:
#@title 🎧 Code Walkthrough: Run 2d Optimizer
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_20_run_2d_optimizer.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def run_optimizer_2d(OptClass, start, lr, steps, **kwargs):
    """Run an optimizer on the Rosenbrock loss surface."""
    x = torch.tensor([start[0]], requires_grad=True, dtype=torch.float)
    y = torch.tensor([start[1]], requires_grad=True, dtype=torch.float)
    opt = OptClass([x, y], lr=lr, **kwargs)

    trajectory = [(x.item(), y.item())]
    losses = []

    for _ in range(steps):
        loss = (1 - x)**2 + 100 * (y - x**2)**2
        loss.backward()
        opt.step()
        opt.zero_grad()
        trajectory.append((x.item(), y.item()))
        losses.append(loss.item())

    return trajectory, losses

# Run all three optimizers
start = (-1.0, 2.0)
steps = 500

results = {}
configs = [
    ('SGD', SGD, 0.0001, {}),
    ('Momentum', SGDMomentum, 0.0001, {'momentum': 0.9}),
    ('Adam', Adam, 0.01, {}),
]

for name, OptClass, lr, kwargs in configs:
    traj, losses = run_optimizer_2d(OptClass, start, lr, steps, **kwargs)
    results[name] = {'trajectory': traj, 'losses': losses}
    final_x, final_y = traj[-1]
    print(f"{name:>10s}: Final ({final_x:>7.3f}, {final_y:>7.3f})  Loss: {losses[-1]:.4f}")

In [ ]:
#@title 🎧 What to Look For: Visualize 2d Race
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_21_visualize_2d_race.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Visualize the optimizer race on the contour plot
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Contour plot with trajectories
ax = axes[0]
x = np.linspace(-2, 2, 200)
y = np.linspace(-1, 3, 200)
X, Y = np.meshgrid(x, y)
Z = np.log1p(loss_fn(X, Y))
ax.contour(X, Y, Z, levels=40, cmap='viridis', alpha=0.6)

colors = {'SGD': '#e74c3c', 'Momentum': '#3498db', 'Adam': '#2ecc71'}
for name, data in results.items():
    traj = np.array(data['trajectory'])
    ax.plot(traj[:, 0], traj[:, 1], '-o', color=colors[name], label=name,
            markersize=1, linewidth=1.5, alpha=0.8)

ax.plot(1, 1, 'k*', markersize=15, zorder=5, label='Minimum')
ax.plot(start[0], start[1], 'ko', markersize=8, zorder=5)
ax.set_xlabel('x', fontsize=12)
ax.set_ylabel('y', fontsize=12)
ax.set_title('Optimizer Paths on Rosenbrock Surface', fontsize=13, fontweight='bold')
ax.legend(fontsize=11)

# Loss curves
ax = axes[1]
for name, data in results.items():
    ax.plot(data['losses'], label=name, linewidth=2, color=colors[name])

ax.set_xlabel('Step', fontsize=12)
ax.set_ylabel('Loss (log scale)', fontsize=12)
ax.set_yscale('log')
ax.set_title('Loss vs Training Step', fontsize=13, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('optimizer_race_2d.png', dpi=150, bbox_inches='tight')
plt.show()
print("Checkpoint: Adam converges fastest by adapting learning rates per-parameter.")

In [ ]:
#@title 🎧 Transition: Real Neural Network Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_22_real_neural_network_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 7. Training and Results: Real Neural Network

Let us test our optimizers on an actual neural network.

In [ ]:
#@title 🎧 What to Look For: Train Nn
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_23_train_nn.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
import torch.nn as nn

# Create a simple regression problem
torch.manual_seed(42)
X = torch.linspace(-3, 3, 200).unsqueeze(1)
y = torch.sin(X) + 0.1 * torch.randn_like(X)

class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(1, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
    def forward(self, x):
        return self.net(x)

def train_net(OptClass, lr, kwargs, epochs=200):
    model = SimpleNet()
    opt = OptClass(model.parameters(), lr=lr, **kwargs)
    losses = []
    for epoch in range(epochs):
        pred = model(X)
        loss = nn.MSELoss()(pred, y)
        loss.backward()
        opt.step()
        opt.zero_grad()
        losses.append(loss.item())
    return losses, model

# Compare optimizers on the neural network
nn_results = {}
nn_configs = [
    ('SGD', SGD, 0.01, {}),
    ('Momentum', SGDMomentum, 0.01, {'momentum': 0.9}),
    ('Adam', Adam, 0.01, {}),
    ('PyTorch Adam', torch.optim.Adam, 0.01, {}),
]

fig, ax = plt.subplots(figsize=(10, 5))
for name, OptClass, lr, kwargs in nn_configs:
    losses, model = train_net(OptClass, lr, kwargs)
    ax.plot(losses, label=name, linewidth=2)
    print(f"{name:>15s}: Final loss = {losses[-1]:.6f}")

ax.set_xlabel('Epoch', fontsize=12)
ax.set_ylabel('MSE Loss', fontsize=12)
ax.set_title('Optimizer Comparison on Neural Network (sin fitting)', fontsize=13, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(alpha=0.3)
ax.set_yscale('log')

plt.tight_layout()
plt.savefig('optimizer_nn.png', dpi=150, bbox_inches='tight')
plt.show()
print("Our custom Adam matches PyTorch's implementation!")

In [ ]:
#@title 🎧 Listen: Final Output Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_24_final_output_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 8. Final Output

In [ ]:
#@title 🎧 What to Look For: Summary Visualization
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_25_summary_visualization.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Summary: Weight update magnitudes over training
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

theta_vals = {'SGD': [], 'Momentum': [], 'Adam': []}
grads_sequence = np.sin(np.linspace(0, 4*np.pi, 100)) + 0.5  # Oscillating + bias

for OptClass, name, lr, kwargs in [
    (SGD, 'SGD', 0.01, {}),
    (SGDMomentum, 'Momentum', 0.01, {'momentum': 0.9}),
    (Adam, 'Adam', 0.01, {}),
]:
    x = torch.tensor([5.0], requires_grad=True)
    opt = OptClass([x], lr=lr, **kwargs)

    for g_val in grads_sequence:
        x.grad = torch.tensor([g_val])
        opt.step()
        theta_vals[name].append(x.item())
        x.grad = None

for ax, (name, vals) in zip(axes, theta_vals.items()):
    ax.plot(vals, linewidth=1.5, color=colors.get(name, '#333'))
    ax.set_title(name, fontsize=13, fontweight='bold')
    ax.set_xlabel('Step')
    ax.set_ylabel('Parameter Value')
    ax.grid(alpha=0.3)

plt.suptitle('How Each Optimizer Responds to Oscillating Gradients', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('optimizer_behavior.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nSummary:")
print("  SGD: Follows every gradient exactly -- noisy and slow")
print("  Momentum: Smooths oscillations, accelerates in consistent directions")
print("  Adam: Adapts per-weight learning rates based on gradient history")
print("\nAdam (and AdamW) is the standard optimizer for modern LLM training.")

In [ ]:
#@title 🎧 Wrap-Up: Closing
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_26_closing.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 9. Reflection and Next Steps

**What we built:** Three optimizers from scratch -- SGD, Momentum, and Adam -- and verified they match PyTorch's implementations.

**Key takeaways:**
- SGD is the foundation: $\theta_{t+1} = \theta_t - \eta \nabla L$
- Momentum adds acceleration by tracking past gradients
- Adam combines momentum with per-weight adaptive learning rates
- Bias correction in Adam prevents early training instability
- AdamW decouples weight decay from the gradient update

**What is next:** In Notebook 4, we will build learning rate schedulers (warmup + cosine decay) and gradient clipping -- the safety mechanisms that prevent training from diverging.